In [ ]:
import mapsims
import numpy as np
import healpy as hp
import pysm3.units as u
from pathlib import Path
%matplotlib inline

# Access existing foreground and CMB map based simulations

The most important repository for Simons Observatory map based simulations is:

* https://github.com/simonsobs/map_based_simulations

It contains all the documentation about available released map and software, in particular:

* Check for news about available releases on the [Logbook](https://github.com/simonsobs/map_based_simulations/blob/master/LOGBOOK.md)
* Check documentation about a release and the path at NERSC, for example the last foreground/CMB [201906_highres_foregrounds_extragalactic_tophat](https://github.com/simonsobs/map_based_simulations/tree/master/201906_highres_foregrounds_extragalactic_tophat)
* Check all the configuration files used to produce a dataset

## Access channel metadata with `mapsims`

In [ ]:
channels = mapsims.parse_channels("tube:ST0")

In [ ]:
print(channels)

In [ ]:
channel = channels[0][0]

In [ ]:
channel.beam

Check the documentation of [`astropy.units`](https://docs.astropy.org/en/stable/units/) for details about handling units.

In [ ]:
channel.center_frequency

In [ ]:
channel.beam.to(u.deg)

In [ ]:
for ch in mapsims.parse_channels("telescope:SA"):
    print(ch)

In [ ]:
mapsims.parse_channels("030", instrument_parameters="planck_deltabandpass")

In [ ]:
for ch in mapsims.parse_channels("all") + mapsims.parse_channels("all", instrument_parameters="planck_deltabandpass"):
    print(ch.telescope, ch, ch.beam, ch.center_frequency)

## Small digression about units

PySM 3 also adds the capability of converting between $K_{CMB}$, $K_{RJ}$ and $MJ/sr$

In [ ]:
(100 * u.uK_CMB).to(u.uK_RJ, equivalencies=u.cmb_equivalencies(channel.center_frequency))

In [ ]:
(100 * u.uK_CMB).to(u.MJy/u.sr, equivalencies=u.cmb_equivalencies(channel.center_frequency))

# Load maps from a map based release at NERSC

In [ ]:
folder = "/project/projectdirs/sobs/v4_sims/mbs/201906_highres_foregrounds_extragalactic_tophat"

In [ ]:
!ls $folder/512

In [ ]:
filename_template = "{nside}/{content}/{num:04d}/simonsobs_{content}_uKCMB_{telescope}{band}_nside{nside}_{num:04d}.fits"

In [ ]:
filename = Path(folder) / filename_template.format(nside=512, content="combined", num=0, 
                                                   telescope=channel.telescope.lower(), band=channel.band)

In [ ]:
print(filename)

In [ ]:
m = hp.read_map(filename, (0,1,2))

In [ ]:
m_P = np.sqrt(m[1]**2 + m[2]**2)

In [ ]:
hp.mollview(m_P, min=0, max=300, title="Polarization amplitude: " + channel.tag, unit="uK_CMB")

# Apply standard galactic masks

Map-based releases also include standard Galactic masks that can be used for analysis,
see the documentation.

In [ ]:
!ls /project/projectdirs/sobs/v4_sims/mbs/201904_highres_foregrounds_equatorial/512/masks

In [ ]:
mask_filename = "mask_equatorial_pol_thr4p0_fsky0p71_ns512.fits"

In [ ]:
mask = hp.read_map("/project/projectdirs/sobs/v4_sims/mbs/201904_highres_foregrounds_equatorial/512/masks/"+mask_filename,
                  dtype=np.bool)

In [ ]:
mask

In [ ]:
hp.mollview(
    mask
, coord="CG")

In [ ]:
m_P_masked = m_P.copy()
m_P_masked[np.logical_not(mask)] = hp.UNSEEN

In [ ]:
hp.mollview(m_P_masked)

# Apply a scanning strategy cut

The `mapsims.SONoiseSimulator` class, which is used to simulate noise, can also automatically download hitmaps that were simulated by TOAST in time domain.

In [ ]:
noise = mapsims.SONoiseSimulator(nside=512)

In [ ]:
hitmaps, ave_nhits = noise.get_hitmaps(tube=channel.tube)

For this simulation set the hitmaps are the same for both channels but in principle they might be different.

In [ ]:
(hitmaps[0] - hitmaps[1]).sum()

In [ ]:
m_P_survey = m_P.copy()
m_P_survey[hitmaps[0] == 0] = hp.UNSEEN

In [ ]:
hp.mollview(m_P_survey)